## In Deep Learning
- Many layers: compositionality
- Convolutions: locality + stationarity of images
- Pooling: Invariance of object class to translations

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# to count the number of parameters
def get_n_params(model):
    np = 0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Loading the MNIST Dataset

We will use torch functionalities to download the data easily along with some minor pre-processing functions.

In [9]:
input_size = 28 * 28
output_size = 10

train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train = True, download = True, 
                                                            transform = transforms.Compose([
                                                                transforms.ToTensor(),
                                                                transforms.Normalize((0.1307,), (0.3081,))
                                                            ])),
                                            batch_size = 64, shuffle = True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train = False, 
                                                            transform = transforms.Compose([
                                                                transforms.ToTensor(),
                                                                transforms.Normalize((0.1307,), (0.3081,))
                                                            ])),
                                            batch_size = 1000, shuffle = True)